# Introduction

This IPython notebook represents my written report and source code for COMPSCI 369 Assignment 3. To generate a PDF version of this report, simply type `rake` at the command line. The generated report can be found at `report.pdf`. Please note that you will need the following software installed.

* Graphviz
* IPython 3
* \LaTeX
* pandoc
* Rake

In [ ]:
from IPython.display import SVG
import itertools as it
import math
import random
import subprocess

# Question 1

In [ ]:
hmm = b'''\
digraph {
    H -> H [label = 0.9375];
    E -> E [label = 0.83333];
    T -> T [label = 0.75];
    H -> E [label = 0.01875];
    H -> T [label = 0.04375];
    E -> H [label = 0.06666];
    E -> T [label = 0.1];
    T -> H [label = 0.125];
    T -> E [label = 0.125];
}
'''
SVG(subprocess.check_output(['dot', '-Tsvg'], input=hmm))

|       |**H**|**E**|**T**|
|:-----:|:---:|:---:|:---:|
| **B** | 0.3 | 0.15| 0.4 |
| **I** | 0.5 | 0.55| 0.1 |
| **N** | 0.2 | 0.3 | 0.5 |